In [12]:
# ================================================
# 📌 1. Import Libraries
# ================================================
import pandas as pd
import numpy as np
from cassandra.cluster import Cluster
from datetime import datetime, timedelta
from scipy.signal import find_peaks

# Deep Learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split

# Visualization
import plotly.graph_objects as go


# ================================================
# 📌 2. Connect to Cassandra & Load Data
# ================================================
cluster = Cluster(['127.0.0.1'], port=9042)  # แก้ตาม Cassandra ของคุณ
session = cluster.connect('stock_data')

stock = "AOT"
start_date = datetime.now() - timedelta(days=150)

rows = session.execute(f"""
    SELECT * FROM candlestick_data 
    WHERE symbol = '{stock}'
""")
data = pd.DataFrame(rows)

if data.empty:
    raise ValueError("❌ No data found")

# จัดรูปแบบ DataFrame
data['time'] = pd.to_datetime(data['time'])
data = data[data['time'] >= start_date]
data = data.sort_values('time').reset_index(drop=True)


# ================================================
# 📌 3. Head & Shoulders Detection (Rule-based)
# ================================================
def detect_head_and_shoulders(prices, distance=3, tolerance=0.05):
    """Return True if pattern exists in window"""
    peaks, _ = find_peaks(prices, distance=distance)
    if len(peaks) < 3:
        return False
    for i in range(len(peaks) - 2):
        ls, head, rs = peaks[i], peaks[i+1], peaks[i+2]
        if head > rs or ls > head: 
            continue
        ls_val, head_val, rs_val = prices[ls], prices[head], prices[rs]
        if head_val > ls_val and head_val > rs_val:
            if abs(ls_val - rs_val) / max(ls_val, rs_val) < tolerance:
                return True
    return False


# ================================================
# 📌 4. Generate Dataset for CNN
# ================================================
window_size = 40
X, y = [], []
close_prices = data['close_price'].values

for i in range(len(close_prices) - window_size):
    window = close_prices[i:i+window_size]
    label = 1 if detect_head_and_shoulders(window) else 0
    X.append(window)
    y.append(label)

X = np.array(X)
y = np.array(y)

# Reshape for Conv1D input (samples, timesteps, features)
X = X.reshape(-1, window_size, 1)


# ================================================
# 📌 5. Train/Test Split
# ================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# ================================================
# 📌 6. Build CNN Model
# ================================================
model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(window_size, 1)),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


# ================================================
# 📌 7. Train Model
# ================================================
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


# ================================================
# 📌 8. Evaluate Model
# ================================================
loss, acc = model.evaluate(X_test, y_test)
print(f"\n✅ Test Accuracy: {acc:.2f}")


# ================================================
# 📌 9. Find HNS Patterns for Visualization
# ================================================
def find_hns_patterns(prices, distance=3, tolerance=0.05):
    """Return list of (left, head, right) indices if found"""
    peaks, _ = find_peaks(prices, distance=distance)
    patterns = []
    if len(peaks) < 3:
        return patterns
    for i in range(len(peaks) - 2):
        ls, head, rs = peaks[i], peaks[i+1], peaks[i+2]
        if head > rs or ls > head: 
            continue
        ls_val, head_val, rs_val = prices[ls], prices[head], prices[rs]
        if head_val > ls_val and head_val > rs_val:
            if abs(ls_val - rs_val) / max(ls_val, rs_val) < tolerance:
                patterns.append((ls, head, rs))
    return patterns


# ================================================
# 📌 10. Plot Candlestick with Detected Patterns
# ================================================
def plot_chart_with_pattern(df, pattern_points, ticker):
    fig = go.Figure()

    # Candlestick chart
    fig.add_trace(go.Candlestick(
        x=df['time'],
        open=df['open_price'],
        high=df['high_price'],
        low=df['low_price'],
        close=df['close_price'],
        name='Candlestick'
    ))

    # วาดจุด Head & Shoulders
    for (l, h, r) in pattern_points:
        fig.add_trace(go.Scatter(
            x=[df['time'].iloc[l]], y=[df['close_price'].iloc[l]],
            mode='markers+text', name='Left Shoulder',
            marker=dict(color='yellow', size=10),
            text=["Left"], textposition="top center"
        ))
        fig.add_trace(go.Scatter(
            x=[df['time'].iloc[h]], y=[df['close_price'].iloc[h]],
            mode='markers+text', name='Head',
            marker=dict(color='red', size=12),
            text=["Head"], textposition="top center"
        ))
        fig.add_trace(go.Scatter(
            x=[df['time'].iloc[r]], y=[df['close_price'].iloc[r]],
            mode='markers+text', name='Right Shoulder',
            marker=dict(color='green', size=10),
            text=["Right"], textposition="top center"
        ))

        # neckline
        fig.add_shape(
            type='line',
            x0=df['time'].iloc[l], y0=df['close_price'].iloc[l],
            x1=df['time'].iloc[r], y1=df['close_price'].iloc[r],
            line=dict(color='green', dash='dash')
        )

    fig.update_layout(
        title=f'{ticker} - Head and Shoulders Pattern',
        xaxis_title='Date',
        yaxis_title='Price (Baht)',
        template='plotly_white',
        xaxis_rangeslider_visible=False,
        height=600
    )
    fig.show()


# ================================================
# 📌 11. Run Detection + Plot
# ================================================
patterns = find_hns_patterns(data['close_price'].values)
plot_chart_with_pattern(data, patterns, stock)


WARNING	cassandra.cluster:cluster.py:__init__()- Cluster.__init__ called with contact_points specified, but no load_balancing_policy. In the next major version, this will raise an error; please specify a load-balancing policy. (contact_points = ['127.0.0.1'], lbp = None)
WARNING	cassandra.cluster:cluster.py:protocol_downgrade()- Downgrading core protocol version from 66 to 65 for 127.0.0.1:9042. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version
WARNING	cassandra.cluster:cluster.py:protocol_downgrade()- Downgrading core protocol version from 65 to 5 for 127.0.0.1:9042. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version


InvalidRequest: Error from server: code=2200 [Invalid query] message="table candlestick_data does not exist"